## Deep Research

One of the classic cross-business Agentic use cases! This is huge.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">A Deep Research agent is broadly applicable to any business area, and to your own day-to-day activities. You can make use of this yourself!
            </span>
        </td>
    </tr>
</table>

In [10]:
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import asyncio
# import sendgrid
import os
# from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import Dict
from IPython.display import display, Markdown
import brevo_python 
import brevo_python 
from brevo_python.api import transactional_emails_api 
from brevo_python.models.send_smtp_email import SendSmtpEmail 
from brevo_python.models.send_smtp_email_to import SendSmtpEmailTo 
from brevo_python.models.send_smtp_email_sender import SendSmtpEmailSender 

In [18]:
load_dotenv(override=True)

True

## OpenAI Hosted Tools

OpenAI Agents SDK includes the following hosted tools:

The `WebSearchTool` lets an agent search the web.  
The `FileSearchTool` allows retrieving information from your OpenAI Vector Stores.  
The `ComputerTool` allows automating computer use tasks like taking screenshots and clicking.

### Important note - API charge of WebSearchTool

This is costing me 2.5 cents per call for OpenAI WebSearchTool. That can add up to $2-$3 for the next 2 labs. We'll use low cost Search tools with other platforms, so feel free to skip running this if the cost is a concern.

Costs are here: https://platform.openai.com/docs/pricing#web-search

In [4]:
INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

In [7]:
message = "Latest AI Agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))

As of mid-2025, several AI agent frameworks have emerged, each offering unique capabilities for developing intelligent, autonomous systems.

**LangChain** is a Python-based framework that enables developers to build applications powered by large language models (LLMs). It offers modular components for creating and managing workflows, integrated memory for stateful applications, and compatibility with multiple LLMs and APIs. LangChain is ideal for conversational AI, automated research assistants, and document analysis. ([linkedin.com](https://www.linkedin.com/pulse/top-5-frameworks-building-ai-agents-2025-sahil-malhotra-wmisc?utm_source=openai))

**LangGraph** extends LangChain by focusing on managing stateful, branching processes using a graph-based architecture for agent interaction. It supports advanced error handling, complex stateful interactions, and multi-agent workflows, making it suitable for applications requiring extensive data retrieval and knowledge fusion, especially in research settings. ([linkedin.com](https://www.linkedin.com/pulse/ai-agent-frameworks-june-2025-comprehensive-overview-chadi-abi-fadel-wcu5c?utm_source=openai))

**CrewAI** is a Python-based framework tailored for orchestrating collaborative AI agents in complex, multi-step workflows across various domains. It features a role-based architecture, dynamic task planning, real-time performance monitoring, and orchestration of a variety of agents as distinct workers. CrewAI is ideal for complex projects requiring teamwork among agents, such as software development or project management. ([linkedin.com](https://www.linkedin.com/pulse/ai-agent-frameworks-june-2025-comprehensive-overview-chadi-abi-fadel-wcu5c?utm_source=openai))

**AutoGen** developed by Microsoft, specializes in orchestrating multiple AI agents to form autonomous, event-driven systems capable of handling complex, multi-agent tasks seamlessly. It offers multi-agent orchestration, runtime convergence for scalable applications, and is ideal for enterprise-level applications needing event-driven workflows, such as customer service automation. ([linkedin.com](https://www.linkedin.com/pulse/ai-agent-frameworks-june-2025-comprehensive-overview-chadi-abi-fadel-wcu5c?utm_source=openai))

**Amazon Bedrock AgentCore** introduced by AWS, is a platform designed to simplify the development and deployment of advanced AI agents. It includes modular services supporting the full production lifecycle, such as scalable serverless deployment, context management, secure service access, tool integration, and enhanced problem-solving capabilities with languages like JavaScript and Python. AgentCore emphasizes flexibility, allowing developers to use only the components they need, facilitating scalable and secure AI agent development. ([techradar.com](https://www.techradar.com/pro/aws-looks-to-super-charge-ai-agents-with-amazon-bedrock-agentcore?utm_source=openai))

These frameworks represent the forefront of AI agent development, each contributing to the evolution of intelligent, autonomous systems across various industries. 

### As always, take a look at the trace

https://platform.openai.com/traces

### We will now use Structured Outputs, and include a description of the fields

In [8]:
# See note above about cost of WebSearchTool

HOW_MANY_SEARCHES = 3

INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."

# Use Pydantic to define the Schema of our response - this is known as "Structured Outputs"
# With massive thanks to student Wes C. for discovering and fixing a nasty bug with this!

class WebSearchItem(BaseModel):
    reason: str = Field(description="Your reasoning for why this search is important to the query.")

    query: str = Field(description="The search term to use for the web search.")


class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description="A list of web searches to perform to best answer the query.")


planner_agent = Agent(
    name="PlannerAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=WebSearchPlan,
)

In [9]:

message = "Latest AI Agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(planner_agent, message)
    print(result.final_output)

searches=[WebSearchItem(reason='To find updated information on AI agent frameworks that are popular or emerging in 2025.', query='latest AI agent frameworks 2025'), WebSearchItem(reason='To explore specific frameworks and their functionalities, including tools and technologies in AI agent development.', query='AI agent development frameworks 2025'), WebSearchItem(reason='To gather insights on industry trends and popular choices in AI agent frameworks within the tech community for 2025.', query='trends in AI agent frameworks 2025')]


In [11]:
@function_tool
def send_email(body: str):
    """ Send out an email with the given subject and HTML body """
    configuration = brevo_python.Configuration()
    api_key = os.environ.get('BREVO_API_KEY')
    if not api_key:
        raise ValueError("BREVO_API_KEY environment variable not set.")
    configuration.api_key['api-key'] = api_key
    api_instance = transactional_emails_api.TransactionalEmailsApi(brevo_python.ApiClient(configuration))
    sender = SendSmtpEmailSender(email="oliver@oliverdreger.cloud", name="Oliver Dreger")
    to = [SendSmtpEmailTo(email="oliver.dreger@gmail.com", name="Oliver Dreger")]
    send_smtp_email = SendSmtpEmail(    
        sender=sender,
        to=to,
        subject="Sales email",
        text_content=body
    )
    try:
        api_response = api_instance.send_transac_email(send_smtp_email)
        print("Email sent successfully!")
        print(f"Response: {api_response}")
    except brevo_python.ApiException as e:
        print(f"Exception when calling TransactionalEmailsApi->send_transac_email: {e}\n")

In [35]:
# @function_tool
# def send_email(subject: str, html_body: str) -> Dict[str, str]:
#     """ Send out an email with the given subject and HTML body """
#     sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
#     from_email = Email("ed@edwarddonner.com") # Change this to your verified email
#     to_email = To("ed.donner@gmail.com") # Change this to your email
#     content = Content("text/html", html_body)
#     mail = Mail(from_email, to_email, subject, content).get()
#     response = sg.client.mail.send.post(request_body=mail)
#     return {"status": "success"}

In [12]:
send_email

FunctionTool(name='send_email', description='Send out an email with the given subject and HTML body', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10cb6f600>, strict_json_schema=True, is_enabled=True)

In [13]:
INSTRUCTIONS = """You are able to send a nicely formatted HTML email based on a detailed report.
You will be provided with a detailed report. You should use your tool to send one email, providing the 
report converted into clean, well presented HTML with an appropriate subject line."""

email_agent = Agent(
    name="Email agent",
    instructions=INSTRUCTIONS,
    tools=[send_email],
    model="gpt-4o-mini",
)



In [14]:
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)


class ReportData(BaseModel):
    short_summary: str = Field(description="A short 2-3 sentence summary of the findings.")

    markdown_report: str = Field(description="The final report")

    follow_up_questions: list[str] = Field(description="Suggested topics to research further")


writer_agent = Agent(
    name="WriterAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)

### The next 3 functions will plan and execute the search, using planner_agent and search_agent

In [15]:
async def plan_searches(query: str):
    """ Use the planner_agent to plan which searches to run for the query """
    print("Planning searches...")
    result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

async def perform_searches(search_plan: WebSearchPlan):
    """ Call search() for each item in the search plan """
    print("Searching...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    return results

async def search(item: WebSearchItem):
    """ Use the search agent to run a web search for each item in the search plan """
    input = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output

### The next 2 functions write a report and email it

In [16]:
async def write_report(query: str, search_results: list[str]):
    """ Use the writer agent to write a report based on the search results"""
    print("Thinking about report...")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing report")
    return result.final_output

async def send_email(report: ReportData):
    """ Use the email agent to send an email with the report """
    print("Writing email...")
    result = await Runner.run(email_agent, report.markdown_report)
    print("Email sent")
    return report

### Showtime!

In [17]:
query ="Latest AI Agent frameworks in 2025"

with trace("Research trace"):
    print("Starting research...")
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    await send_email(report)  
    print("Hooray!")




Starting research...
Planning searches...
Will perform 3 searches
Searching...
Finished searching
Thinking about report...
Finished writing report
Writing email...
Email sent successfully!
Response: {'message_id': '<202507272035.93916604807@smtp-relay.mailin.fr>',
 'message_ids': None}
Email sent
Hooray!


### As always, take a look at the trace

https://platform.openai.com/traces

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../../assets/thanks.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00cc00;">Congratulations on your progress, and a request</h2>
            <span style="color:#00cc00;">You've reached an important moment with the course; you've created a valuable Agent using one of the latest Agent frameworks. You've upskilled, and unlocked new commercial possibilities. Take a moment to celebrate your success!<br/><br/>Something I should ask you -- my editor would smack me if I didn't mention this. If you're able to rate the course on Udemy, I'd be seriously grateful: it's the most important way that Udemy decides whether to show the course to others and it makes a massive difference.<br/><br/>And another reminder to <a href="https://www.linkedin.com/in/eddonner/">connect with me on LinkedIn</a> if you wish! If you wanted to post about your progress on the course, please tag me and I'll weigh in to increase your exposure.
            </span>
        </td>
    </tr>

--- 

# Notebook Summary by Gemini

---

Of course. Here is a summary of the Jupyter Notebook `/Users/oliverdreger/Documents/mygit/GitHub/agents/2_openai/community_contributions/4_lab4_oliver.ipynb`.

# Summary of Notebook 

<img src="../../assets/agentic_workflow.png" width="25%">
<p style="text-align: left; font-size: 12px; color: #666; margin-top: 5px;"><em>Source: Copilot</em></p>

This Jupyter Notebook demonstrates how to build a sophisticated, multi-agent system to perform deep research on a given topic, synthesize the findings into a detailed report, and email the result. The notebook illustrates a classic agentic workflow by decomposing a complex task into a series of smaller, specialized tasks handled by different agents.

The core of the notebook is the creation and orchestration of four distinct agents:

1.  **`planner_agent`**: This agent is responsible for the initial planning phase. Given a research query, it uses a structured output (`WebSearchPlan` Pydantic model) to generate a list of several targeted web search queries. This ensures the research is comprehensive and well-structured from the start.

2.  **`search_agent`**: This is a simple research assistant that takes a single search term and uses the `WebSearchTool` to find information. It then produces a concise, point-form summary of the results, designed to be easily consumed by the next agent in the chain.

3.  **`writer_agent`**: This agent acts as a senior researcher. It receives the original query and the collection of summaries from the `search_agent`. Its task is to synthesize this information into a cohesive, detailed, and lengthy report (targeting over 1000 words). It also uses a structured output (`ReportData` Pydantic model) to return not only the markdown report but also a short summary and a list of suggested follow-up questions.

4.  **`email_agent`**: The final agent in the workflow. It takes the generated report and uses a custom `send_email` tool (which integrates with the SendGrid API) to format the report into a clean HTML email and send it.

The notebook then defines a series of `async` functions to orchestrate the workflow:
*   `plan_searches()`: Calls the `planner_agent` to create the research plan.
*   `perform_searches()`: Executes all the planned searches in parallel using `asyncio.gather` and the `search_agent`.
*   `write_report()`: Feeds the search results to the `writer_agent` to generate the final report.
*   `send_email()`: Uses the `email_agent` to send the completed report.

The final "Showtime!" cell chains these functions together, demonstrating the complete, end-to-end automated process: from receiving a simple query to planning the research, executing it, writing a comprehensive report, and emailing it.

Key concepts highlighted in this notebook include **agent decomposition**, the use of **structured outputs** with Pydantic for reliable inter-agent communication, and **asynchronous execution** for efficient parallel processing.

--- 

# Transcript Summary by Gemini

---

## Day 4 - Building Deep Research Agents: Implementing OpenAI's Web Search Tool
A summary of the course transcript for "Day 4 - Building Deep Research Agents: Implementing OpenAI's Web Search Tool" is as follows:

The course on day four focuses on building a deep research agent, a common application of Agentic AI. This project involves creating an agent capable of searching the internet to research a given topic. This is similar to the deep research functionalities offered by frontier labs like OpenAI. The project will utilize previously learned concepts such as tools and structured outputs, and will also introduce hosted tools, which are run remotely by OpenAI. While the lab will be conducted in a notebook format for experimentation, the following day will transition to using Python modules.

A significant new element introduced is the use of OpenAI's hosted tools. There are three such tools available: the web search tool, the file search tool for querying vector stores, and a computer tool for tasks like taking screenshots. This session will focus on the web search tool. The instructor notes that this tool is not inexpensive, costing around 2.5 cents per call with the cheapest model. A deep research task involving multiple searches could quickly accumulate costs. To manage this, users can monitor their spending and adjust settings. An alternative, more cost-effective method will be introduced in later weeks.

The first agent created in this project is the "search agent." Its system prompt, taken directly from OpenAI's documentation, instructs it to act as a research assistant that searches the web for a given term and provides a concise two to three-paragraph summary of the main points. This agent is configured to use the web search tool, and its use is made mandatory. The cost and performance of the agent can be managed by selecting the search context size (low, medium, or high) and the model (e.g., GPT-4 mini for a lower cost).

The instructor then demonstrates running the search agent with the query "latest AI agent frameworks in 2025." The results, presented in markdown, include a list of frameworks such as LangChain, LangGraph, Crew, and Autogen. The instructor comments on the mixed accuracy of the results, noting that some listed items aren't strictly agent frameworks. Finally, the process of examining the trace of the agent's execution in the OpenAI platform is shown, confirming that the web search tool was used as intended.

## Day 4 - Building a Planner Agent: Using Structured outputs with Pydantic in AI 

This is a summary of the course transcript for "Day 4 - Building a Planner Agent: Using Structured outputs with Pydantic in AI":

The focus of this section of the course is on creating a "planner agent." This agent's role is to take a user's query and generate a set of web searches to conduct thorough research on the topic. To manage costs associated with OpenAI's search tool, the number of searches is initially limited to three, though this can be adjusted.

The system prompt for the planner agent is straightforward: "You are a helpful research assistant. Given a query, come up with a set of web searches to perform to best answer that query output. And then the number three terms to query for."

A key concept in building this agent is the use of structured outputs with the Pydantic library. This is achieved by defining a Python class, `WebSearchItem`, which inherits from Pydantic's `BaseModel`. This class acts as a schema for the desired output and includes two string fields: `reason` and `query`. The docstrings for these fields are important as they provide context to the model, helping it to populate them correctly.

The instructor highlights a technique similar to "chain of thought" prompting: by asking the model to first generate a `reason` for a search before the `query` itself, the model is encouraged to "think through" its response, which often leads to higher-quality and more coherent search queries. It is emphasized that this is a result of the model's next-token prediction process, not genuine cognition.

Another Pydantic class, `WebSearchPlan`, is defined to contain a single field, `searches`, which is a list of `WebSearchItem` objects.

The planner agent is then created, using "gpt-4-mini" as the model. Crucially, the `output_type` for this agent is set to the `WebSearchPlan` class. This instructs the agent to return its output as a structured object conforming to this schema, rather than as plain text. The instructor clarifies that behind the scenes, this process involves converting the Pydantic model into a JSON schema that the language model is prompted to adhere to.

Finally, a demonstration is shown where the planner agent is given the query "latest AI agent frameworks in 2025." The agent successfully returns a `WebSearchPlan` object. This object contains a list of three `WebSearchItem` objects, each with a specific `reason` and a corresponding `query` to be used for searching.

## Day 4 - Building an End-to-End Research Pipeline with GPT-4 Agents & Async Tasks 

Here is a summary of the course transcript for "Day 4 - Building an End-to-End Research Pipeline with GPT-4 Agents & Async Tasks":

This section of the course focuses on assembling the previously built components into a complete, end-to-end research pipeline. This pipeline will automate the entire process from receiving a query to planning searches, executing them, writing a report, and finally, emailing the result.

The process begins by reintroducing a `send_email` tool, which is created by converting a Python function into an agent tool using the `@function_tool` decorator. This tool is then provided to a new `email_agent`. This agent is given the instructions and autonomy to take a detailed report, convert it into a well-presented HTML email, and create its own subject line before sending it.

Next, a "researcher" or "writer" agent is created. This senior agent's task is to take the original query and the initial research gathered by other agents and synthesize it into a cohesive, lengthy, and detailed report in Markdown. A key feature of this agent is its use of structured outputs. It is configured to return a Pydantic `ReportData` object, which has three fields: a `short_summary`, the `full_markdown_final_report`, and `follow_up_suggestions`.

The core of the lesson is the "crunch time" section, where five asynchronous functions are defined to orchestrate the entire workflow:

1.  **`plan_searches`**: This function calls the `planner_agent` to generate a list of web searches based on the initial query.
2.  **`perform_searches`**: This function takes the search plan and executes all the individual searches *in parallel*. This is a crucial efficiency step achieved using Python's `asyncio.gather`, which runs multiple instances of the `search_agent` concurrently.
3.  **`search_for_item`**: This is the helper function called by `perform_searches` for each individual search, invoking the `search_agent`.
4.  **`write_report`**: This function calls the `writer_agent`, providing it with the query and the collected search results to generate the final structured `ReportData` object.
5.  **`send_email`**: This final function calls the `email_agent` to format and send the generated report.

The lesson culminates in the execution block where these asynchronous functions are `await`ed in sequence, demonstrating the complete pipeline in action. The flow starts with a query, plans the necessary searches, performs them simultaneously, synthesizes the results into a detailed report, and concludes by sending the final report as an HTML email.

## Day 4 - Building a Deep Research Agent: Parallel Searches with AsyncIO

Here is a summary of the transcript "Day 4 - Building a Deep Research Agent: Parallel Searches with AsyncIO":

The session begins by confirming the successful completion of the deep research pipeline that was built. The initial run, configured to perform three searches, finished successfully and sent a well-formatted HTML email. This email contained a good introduction, a detailed analysis of AI agent frameworks (mentioning "Crew" at the top), and functional reference links at the bottom. The instructor expresses satisfaction with building a powerful deep research framework with surprisingly minimal code, highlighting its potential for expansion.

To demonstrate this potential, the instructor scales up the operation by increasing the number of searches from three to twenty. This change is made in the `planner_agent`'s instructions. After re-running the entire pipeline, the new, more extensive research is completed. The resulting email report, while identifying similar key frameworks, is significantly more substantive. It includes more detailed information, such as the applications and benefits associated with each framework, a section on commercial implications, and a concluding summary.

A key takeaway highlighted is the efficiency gained through parallel processing. By examining the execution trace, the instructor shows that after the initial `planner_agent` ran, all twenty `search_agent` tasks were executed simultaneously thanks to Python's `asyncio`. This parallel execution was followed by the sequential execution of the `writer_agent` and the `email_agent` to synthesize the report and send the final email.

The lesson concludes with the instructor encouraging students to think about how they could further enhance and build upon this deep research agent. It is also mentioned that the next session will focus on packaging this entire framework into a standalone, take-away application.